In [1]:
import csv
import json
import os
from google.cloud import storage
from urllib.parse import urlparse

In [2]:
def csv_to_jsonl(input_path, output_path):
    """
    CSV 파일을 읽어 JSONL 파일로 변환하는 함수
    
    Args:
        input_path (str): 입력 CSV 파일 경로
        output_path (str): 출력 JSONL 파일 경로
    """
    try:
        with open(input_path, mode='r', encoding='utf-8-sig') as f_in, \
             open(output_path, mode='w', encoding='utf-8') as f_out:
            
            # CSV 파일을 딕셔너리 형태로 읽기
            reader = csv.DictReader(f_in)
            
            for row in reader:
                # 각 행을 JSON 문자열로 변환 (한글 깨짐 방지를 위해 ensure_ascii=False)
                json_line = json.dumps(row, ensure_ascii=False)
                f_out.write(json_line + '\n')
                
        print(f"변환 완료: {output_path}")
        
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {input_path}")
    except Exception as e:
        print(f"오류 발생: {e}")

In [3]:
def download_from_gcs(gcs_uri, local_filename):
    """
    gs:// URI를 사용하여 GCS에서 파일을 다운로드하는 함수
    
    Args:
        gcs_uri (str): 다운로드할 파일의 GCS 경로 (예: gs://my-bucket/data.csv)
        local_filename (str): 저장할 로컬 파일 경로
    """
    try:
        # 1. URI 파싱 (버킷명과 파일 경로 분리)
        parsed = urlparse(gcs_uri)
        if parsed.scheme != 'gs':
            raise ValueError("GCS URI는 'gs://'로 시작해야 합니다.")
            
        bucket_name = parsed.netloc
        blob_name = parsed.path.lstrip('/') # 앞의 '/' 제거

        # 2. GCS 클라이언트 및 버킷/블롭 객체 생성
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)

        # 3. 다운로드 실행
        print(f"다운로드 시작: {gcs_uri} -> {local_filename}")
        blob.download_to_filename(local_filename)
        print("다운로드 완료!")

    except Exception as e:
        print(f"오류 발생: {e}")

In [4]:
def upload_to_gcs(local_file_path, gcs_dest_uri):
    """
    로컬 파일을 gs:// URI 경로의 GCS로 업로드하는 함수
    
    Args:
        local_file_path (str): 업로드할 로컬 파일 경로
        gcs_dest_uri (str): 업로드될 GCS 목적지 경로 (예: gs://my-bucket/folder/data.jsonl)
    """
    try:
        # 1. 로컬 파일 존재 여부 확인
        if not os.path.isfile(local_file_path):
            raise FileNotFoundError(f"로컬 파일을 찾을 수 없습니다: {local_file_path}")

        # 2. URI 파싱 (버킷명과 저장될 경로 분리)
        parsed = urlparse(gcs_dest_uri)
        if parsed.scheme != 'gs':
            raise ValueError("GCS URI는 'gs://'로 시작해야 합니다.")
            
        bucket_name = parsed.netloc
        blob_name = parsed.path.lstrip('/') # 앞의 '/' 제거하여 상대 경로로 만듦

        # 3. GCS 클라이언트 및 업로드 설정
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)

        # 4. 업로드 실행
        print(f"업로드 시작: {local_file_path} -> {gcs_dest_uri}")
        blob.upload_from_filename(local_file_path)
        print("업로드 완료!")

    except Exception as e:
        print(f"오류 발생: {e}")

In [5]:
if __name__ == "__main__":
    # 여기에 다운로드할 GCS 경로와 저장할 파일명을 입력하세요
    GCS_PATH = "gs://kevin-vertexai-002/expense.csv"
    LOCAL_PATH = "expense.csv"
    JSONL_PATH = "expense.jsonl"
    JSONL_GCS_PATH = "gs://kevin-vertexai-002/expense.jsonl"
    
    download_from_gcs(GCS_PATH, LOCAL_PATH)
    
    csv_to_jsonl(LOCAL_PATH, JSONL_PATH) 
    
    upload_to_gcs(JSONL_PATH, JSONL_GCS_PATH)

다운로드 시작: gs://kevin-vertexai-002/expense.csv -> expense.csv
다운로드 완료!
변환 완료: expense.jsonl
업로드 시작: expense.jsonl -> gs://kevin-vertexai-002/expense.jsonl
업로드 완료!
